In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

def normalize(x, m, s):
    return (x-m)/s #subtract mean and divide by std dev

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

We want this mean and std dev to be 0 and 1. Why? We will find out as we go along

In [5]:
x_train = normalize(x_train, train_mean, train_std)
# NB: use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(3.0614e-05), tensor(1.))

In [7]:
#export
def test_near_zero(a, tol=1e-3):
    assert a.abs()<tol, f'Near zero: {a}'

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [9]:
n, m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

# Foundations version

## Basic architecture

In [10]:
# num hidden
nh = 50

To simplify things for now, we'll use 1 final activation and MSE as loss

architecture:

input: 50000 * 784 \
layer 1: 784 * 50 \
layer 2: 50 * 1

The inputs are now mean 0 and std dev 1 because we normalized them. So the input to 1st layer are mean 0 and std dev 1. We want that for inputs to 2nd layer as well.

In [11]:
# simplified kaiming init / he init
w1 = torch.randn(m, nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [12]:
# non-kaiming (random)
w1r = torch.randn(m, nh)
b1r = torch.zeros(nh)
w2r = torch.randn(nh, 1)
b2r = torch.zeros(1)

`randn` gives random numbers with mean 0 and std dev 1. Hence, if we divide by `sqrt(m)`, new std dev will also be `1/sqrt(m)`

In [13]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [14]:
x_valid.mean(), x_valid.std() # this should be mean 0, std 1

(tensor(-0.0058), tensor(0.9924))

In [15]:
#linear layer
def lin(x, w, b):
    return x@w + b

In [16]:
t = lin(x_valid, w1, b1)

In [17]:
t.mean(), t.std()

(tensor(0.1389), tensor(0.9789))

In [18]:
tr = lin(x_valid, w1r, b1r)
tr.mean(), tr.std() # These are very far off from mean 0 std 1

(tensor(-3.2152), tensor(27.9476))

In [19]:
def relu(x):
    return x.clamp_min(0.)

In [20]:
t = relu(lin(x_valid, w1, b1))

In [21]:
# but now - mean 0 and std 1 is not true. True input to next layer are the values from relu
t.mean(), t.std()

(tensor(0.4553), tensor(0.6309))

This is because after only linear layer, we had (0,1), but now all negatives are replaced by 0 so definitely mean is affected. Similarly, std will also now be about half of what it was after `x@w + b`. \
Hence, if we multiply by `2/sqrt(m)` we might be able to get the desired outcome

In [22]:
# kaiming init / he init for relu
w1 = torch.randn(m, nh)*math.sqrt(2.0/m)

In [23]:
w1.mean(), w1.std()

(tensor(-0.0004), tensor(0.0503))

In [24]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.4919), tensor(0.7583))

In [25]:
#export
from torch.nn import init

In [26]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out') #fan_in preserves variance of weights in fwd pass, fan_out preserves variance in bwd pass
t = relu(lin(x_valid, w1, b1))

In [27]:
w1.mean(), w1.std()

(tensor(-0.0001), tensor(0.0505))

In [28]:
t.mean(), t.std()

(tensor(0.4881), tensor(0.7640))

In [29]:
w1.shape

torch.Size([784, 50])

Our weights are 784x50, whereas pytorch's linear layer creates weights as 50x784. That's why using `fan_out` instead of `fan_in` even though we are in fwd pass.

In [30]:
import torch.nn

In [31]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

This is because, in pytorch linear layer (`F.linear`), `output=input.matmul(weight.t())`. Hence it's being multiplied by transpose.

In [32]:
# since we saw relu was taking away all negative and increasing the mean to 0.5,
# what if we decrease it back?
def relu(x):
    return x.clamp_min(0.) - 0.5

In [33]:
# kaiming he init for new relu
w1 = torch.randn(m, nh)*math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t1.std()

(tensor(0.0725), tensor(0.8236))

In [34]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [35]:
%timeit -n 10 _=model(x_valid)

5.26 ms ± 53.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

## Loss function: MSE

In [37]:
model(x_valid).shape

torch.Size([10000, 1])

In [38]:
y_valid.shape

torch.Size([10000])

We need `squeeze()` to get rid of that trailing (,1), in order to use mse. (`mse` is not suitable for classification, just using here for simplicity)

Just writing `squeeze()` will get rid of all unit axes. Hence `squeeze(-1)` is more explicit and will only remove the last unit axis

In [39]:
def mse(output, targ):
    return (output.squeeze(-1) - targ).pow(2).mean()

In [40]:
y_train, y_valid = y_train.float(), y_valid.float()

In [41]:
preds = model(x_train)

In [42]:
preds.shape

torch.Size([50000, 1])

In [43]:
mse(preds, y_train)

tensor(25.5892)

## Gradients and backward pass

In [44]:
def mse_grad(inp, targ):
    # grad of loss w.r.t. output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [45]:
def relu_grad(inp, out):
    # grad of relu w.r.t. input activations
    inp.g = (inp>0).float() * out.g

In [46]:
def lin_grad(inp, out, w, b):
    # grad of matmul w.r.t. input
    inp.g = out.g @ w.t() # gradient of matrix product is matrix product with the transpose
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [47]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp@w1 + b1
    l2 = relu(l1)
    out = l2@w2 + b2
    # we don't actually need the loss in the backward
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [48]:
forward_and_backward(x_train, y_train)

In [49]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

Use PyTorch autograd to check our results

In [50]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [51]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [52]:
loss = forward(xt2, y_train)

In [53]:
loss.backward()

In [54]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig)

## Refactor model

### Layers as classes

In [55]:
class Lin():
    def __init__(self, w, b):
        self.w, self.b = w, b
    
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        # self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.w.g = torch.einsum('bi,bj->ij', self.inp, self.out.g)
        self.b.g = self.out.g.sum(0)

In [56]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - 0.5
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp>0).float() * self.out.g

In [57]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [58]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [59]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [60]:
%time loss = model(x_train, y_train)

CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 29.5 ms


In [61]:
%time model.backward()

CPU times: user 204 ms, sys: 64 ms, total: 268 ms
Wall time: 137 ms


In [62]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## Module.forward()

In [63]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self):
        raise Exception('not implemented')
    def backward(self):
        self.bwd(self.out, *self.args)

In [64]:
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.) - 0.5
    def bwd(self, out, inp):
        inp.g = (inp>0).float() * out.g

In [65]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b
        
    def forward(self, inp):
        return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum('bi,bj->ij', inp, out.g)
        self.b.g = out.g.sum(0)

In [66]:
class Mse(Module):
    def forward(self, inp, targ):
        return (inp.squeeze() - targ).pow(2).mean()
    
    def bwd(self, out, inp, targ):
        inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]

In [67]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [68]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model()

In [69]:
%time loss = model(x_train, y_train)

CPU times: user 56 ms, sys: 4 ms, total: 60 ms
Wall time: 29.1 ms


In [70]:
%time model.backward()

CPU times: user 208 ms, sys: 60 ms, total: 268 ms
Wall time: 134 ms


In [71]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## Without einsum

In [72]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b
        
    def forward(self, inp):
        return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [73]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model()

In [74]:
%time loss = model(x_train, y_train)

CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 29.3 ms


In [75]:
%time model.backward()

CPU times: user 200 ms, sys: 64 ms, total: 264 ms
Wall time: 132 ms


In [76]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## nn.Linear and nn.Module

In [77]:
#export
from torch import nn

In [78]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse
    
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x.squeeze(), targ)

In [79]:
model = Model(m, nh, 1)

In [80]:
%time loss = model(x_train, y_train)

CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 31 ms


In [81]:
%time loss.backward()

CPU times: user 84 ms, sys: 0 ns, total: 84 ms
Wall time: 41.6 ms


# Export

In [82]:
!python notebook2script.py 02_fully_connected.ipynb

converted 02_fully_connected.ipynb to nb_02.py
